In [12]:
import mujoco
import mujoco.viewer
import time
import numpy as np

In [21]:
# from scipy.spatial.transform import Rotation as R

m = mujoco.MjModel.from_xml_path("/home/mmlab-rl/codes/SBC1/model/n2/mjcf/N2_contactsite1.xml")
d = mujoco.MjData(m)
end_time = 10.0
with mujoco.viewer.launch_passive(m, d) as viewer:
    # viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONVEXHULL] = 1

    start = time.time()
    mujoco.mj_forward(m, d)
    while viewer.is_running() and time.time() - start < end_time:
        step_start = time.time()

        
        # mujoco.mj_step(m, d)
        # qyorigin += 0.001
        # d.qpos[m.joint("L_leg_hip_yaw_joint").id+6] += 0.001
        # d.qpos[m.joint("L_leg_hip_roll_joint").id+6] += 0.001
        # d.qpos[m.joint("L_leg_hip_pitch_joint").id+6] -= 0.0002
        # 使用雅可比矩阵求逆生成速度
        target_quat = np.array([ 0.922,  0.388, 0, 0])  # 请替换为你的目标四元数

        # 获取base_link到L_leg_hip_pitch_link的当前四元数
        body_id = m.body("L_leg_hip_pitch_link").id
        base_id = m.body("base_link").id

        # mujoco中四元数顺序为[x, y, z, w]
        current_quat = d.xquat[body_id]
        base_quat = d.xquat[base_id]

        # 计算base_link到L_leg_hip_pitch_link的相对四元数
        # r_body = R.from_quat(current_quat)
        # r_base = R.from_quat(base_quat)
        # relative_quat = (r_base.inv() * r_body).as_quat()
        
        base_quat_conj = np.zeros(4)
        relative_quat = np.zeros(4)
        mujoco.mju_negQuat(base_quat_conj, base_quat)
        mujoco.mju_mulQuat(relative_quat, current_quat, base_quat_conj)
        # mujoco.mju_quat2Vel(error_ori, error_quat, 1.0)

        # 计算需要的修正四元数
        # r_target = R.from_quat(target_quat)
        # r_correction = r_target * r_base

        # 计算目标角速度
        # error_quat = R.from_quat(relative_quat).inv() * r_target
        # angular_velocity = 2 * error_quat.as_rotvec()
        
        relative_quat_conj = np.zeros(4)
        error_quat = np.zeros(4)
        error_ori = np.zeros(3)
        mujoco.mju_negQuat(relative_quat_conj, relative_quat)
        mujoco.mju_mulQuat(error_quat, target_quat, relative_quat_conj)
        mujoco.mju_quat2Vel(error_ori, error_quat, 1.0)
        angular_velocity = 2 * error_ori

        # 获取雅可比矩阵
        jacobian = np.zeros((3, m.nv))
        mujoco.mj_jacBody(m, d, None, jacobian, body_id)

        # 计算广义速度
        l_leg_yaw_id = m.joint("L_leg_hip_yaw_joint").id+5
        generalized_velocity = np.linalg.pinv(jacobian[:,l_leg_yaw_id:l_leg_yaw_id+3]) @ angular_velocity

        # 设置速度
        d.qvel[l_leg_yaw_id:l_leg_yaw_id+3] = generalized_velocity
        # print(d.qvel[l_leg_yaw_id:l_leg_yaw_id+3])
        d.qpos[l_leg_yaw_id+1:l_leg_yaw_id+4] += d.qvel[l_leg_yaw_id:l_leg_yaw_id+3] * m.opt.timestep
        
        mujoco.mj_forward(m, d)

        viewer.sync()
        time_until_next_step = m.opt.timestep - (time.time() - step_start)
        if time_until_next_step > 0:
            time.sleep(time_until_next_step)

In [6]:
print(m.joint("L_leg_hip_yaw_joint").id)

5


In [22]:
body_id = m.body("L_leg_hip_pitch_link").id
base_id = m.body("base_link").id

# mujoco中四元数顺序为[x, y, z, w]
current_quat = d.xquat[body_id]
base_quat = d.xquat[base_id]
print("current_quat:", current_quat)
print("base_quat:", base_quat)

current_quat: [ 9.23894515e-01  3.82647259e-01  5.06110101e-06 -1.78938595e-06]
base_quat: [1. 0. 0. 0.]


In [20]:
l_leg_yaw_id

10